# Movielens recommendation hands-on
### Movielens data
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## Project Flow
### 1) Preprocessing
### 2) Data Analysis
### 3) Pick 5 favorite movies and add to 'rating'
### 4) make CSR matrix
### 5) composite model and fit
#### - als_model = AlternatingleastSquares
### 6) pick 
#### - one of my favorite
#### - another
#### - and check out the preference rate the model predicted
### 7) get recommendation of similar movies(similar_items)
### 8) get recommendation of what i may like

In [92]:
import pandas as pd
import os
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
import numpy as np
import re

### 1) Preprocessing

In [93]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [94]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [95]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [96]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 2) Data Analysis
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [97]:
num_unique_movies=ratings['movie_id'].nunique()
print(f'the number of unique movies is {num_unique_movies}')

the number of unique movies is 3628


In [98]:
num_unique_users = ratings['user_id'].nunique()
print(f'the number of unique users is {num_unique_users}')


the number of unique users is 6039


In [99]:
ratings.index

Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            1000198, 1000199, 1000200, 1000201, 1000202, 1000203, 1000205,
            1000206, 1000207, 1000208],
           dtype='int64', length=836478)

In [100]:
movie_count = ratings.groupby('movie_id',axis=0,sort=True)['user_id'].count()
movie_count.sort_values(ascending=False).head(30)
#ratings.groupby('movie_id',axis=0,sort=True)['user_id'].count().sum()

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

### That's not good, **I want to add movie name** for intuitive data grasp..

In [101]:
unique_movies2 =movies['title']
print(unique_movies2)

0                         Toy Story (1995)
1                           Jumanji (1995)
2                  Grumpier Old Men (1995)
3                 Waiting to Exhale (1995)
4       Father of the Bride Part II (1995)
                       ...                
3878               Meet the Parents (2000)
3879            Requiem for a Dream (2000)
3880                      Tigerland (2000)
3881               Two Family House (2000)
3882                 Contender, The (2000)
Name: title, Length: 3883, dtype: object


1. eliminate space and year

In [102]:
# p= re.complie('\([0-9]+\)')
print(len(movies.index))

for i in range(len(movies.index)):
    movies['title'][i] = re.sub(' ', '',movies['title'][i])
    movies['title'][i] = re.sub('\([0-9]+\)', '',movies['title'][i] )
movies.head()

3883


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,movie_id,title,genre
0,1,ToyStory,Animation|Children's|Comedy
1,2,Jumanji,Adventure|Children's|Fantasy
2,3,GrumpierOldMen,Comedy|Romance
3,4,WaitingtoExhale,Comedy|Drama
4,5,FatheroftheBridePartII,Comedy


data mapping

In [103]:
unique_movies=movies['title'].unique()

idx_to_movies = {k:v for k,v in enumerate(unique_movies)}
print(idx_to_movies)

{0: 'ToyStory', 1: 'Jumanji', 2: 'GrumpierOldMen', 3: 'WaitingtoExhale', 4: 'FatheroftheBridePartII', 5: 'Heat', 6: 'Sabrina', 7: 'TomandHuck', 8: 'SuddenDeath', 9: 'GoldenEye', 10: 'AmericanPresident,The', 11: 'Dracula:DeadandLovingIt', 12: 'Balto', 13: 'Nixon', 14: 'CutthroatIsland', 15: 'Casino', 16: 'SenseandSensibility', 17: 'FourRooms', 18: 'AceVentura:WhenNatureCalls', 19: 'MoneyTrain', 20: 'GetShorty', 21: 'Copycat', 22: 'Assassins', 23: 'Powder', 24: 'LeavingLasVegas', 25: 'Othello', 26: 'NowandThen', 27: 'Persuasion', 28: 'CityofLostChildren,The', 29: 'ShanghaiTriad(Yaoayaoyaodaowaipoqiao)', 30: 'DangerousMinds', 31: 'TwelveMonkeys', 32: 'WingsofCourage', 33: 'Babe', 34: 'Carrington', 35: 'DeadManWalking', 36: 'AcrosstheSeaofTime', 37: 'ItTakesTwo', 38: 'Clueless', 39: 'Cry,theBelovedCountry', 40: 'RichardIII', 41: 'DeadPresidents', 42: 'Restoration', 43: 'MortalKombat', 44: 'ToDieFor', 45: 'HowtoMakeanAmericanQuilt', 46: 'Seven(Se7en)', 47: 'Pocahontas', 48: 'WhenNightIsFall

In [104]:
movies_to_idx= {v:k for k,v in enumerate(unique_movies)}
movies_to_idx['Bug\'sLife,A']

2274

In [105]:
ratings=pd.concat([ratings['movie_id'],ratings],axis=1)
ratings.columns

Index(['movie_id', 'user_id', 'movie_id', 'count', 'timestamp'], dtype='object')

In [106]:
# ratings=ratings.rename(columns={"movie_id": ""},level=1)

# ratings.head()
ratings.columns = ['movie_name', 'user_id', 'movie_id', 'count', 'timestamp']
ratings.head()

,movie_name,user_id,movie_id,count,timestamp
0,1193,1,1193,5,978300760
1,661,1,661,3,978302109
2,914,1,914,3,978301968
3,3408,1,3408,4,978300275
4,2355,1,2355,5,978824291


In [107]:
ratings_column=list(ratings.columns)
ratings_column
ratings_column.insert(2, ratings_column.pop(0))
ratings_column

['user_id', 'movie_id', 'movie_name', 'count', 'timestamp']

In [108]:
ratings = ratings[ratings_column]
ratings

,user_id,movie_id,movie_name,count,timestamp
0,1,1193,1193,5,978300760
1,1,661,661,3,978302109
2,1,914,914,3,978301968
3,1,3408,3408,4,978300275
4,1,2355,2355,5,978824291
...,...,...,...,...,...
1000203,6040,1090,1090,3,956715518
1000205,6040,1094,1094,5,956704887
1000206,6040,562,562,5,956704746
1000207,6040,1096,1096,4,956715648


In [109]:
ratings['movie_name'] = ratings['movie_name'].map(idx_to_movies.get)

In [110]:
ratings.head()

,user_id,movie_id,movie_name,count,timestamp
0,1,1193,GoodFellas,5,978300760
1,1,661,Bloodsport2,3,978302109
2,1,914,"Women,The",3,978301968
3,1,3408,ReadytoRumble,4,978300275
4,1,2355,"Hi-LoCountry,The",5,978824291


### Any NaN or duplicated value?

In [111]:
ratings.isnull().sum()

user_id           0
movie_id          0
movie_name    11239
count             0
timestamp         0
dtype: int64

### movie name is not good

In [112]:
ratings.duplicated().sum()

0

#### Now, movie names are added!

### 3) Pick 5 favorite movies and add to 'rating'

a) add id idx

In [113]:
my_id = ratings.tail()['user_id'].iloc[-1]+1
my_id

6041

In [114]:
my_favorite=['Grease','Titanic','Mulan','Bug\'sLife,A','ToyStory']
my_favorite_idx=[movies_to_idx[idx] for idx in my_favorite]
my_playlist=pd.DataFrame({'user_id':[my_id]*5, 'movie_id': my_favorite_idx,'movie_name':my_favorite, 'count':[5]*5})
my_playlist.head()

if not ratings.isin({'user_id':[my_id]})['user_id'].any():
     ratings=ratings.append(my_playlist)
ratings.tail(10)


,user_id,movie_id,movie_name,count,timestamp
1000203,6040,1090,PrerokbeOgnja,3,956715518.0
1000205,6040,1094,Palookaville,5,956704887.0
1000206,6040,562,NakedinNewYork,5,956704746.0
1000207,6040,1096,"Funeral,The",4,956715648.0
1000208,6040,1097,Sleepover,4,956715569.0
0,6041,1356,Grease,5,NaN
1,6041,1668,Titanic,5,NaN
2,6041,1833,Mulan,5,NaN
3,6041,2274,"Bug'sLife,A",5,NaN
4,6041,0,ToyStory,5,NaN


In [115]:
ratings = ratings.reset_index(drop=True)
ratings

,user_id,movie_id,movie_name,count,timestamp
0,1,1193,GoodFellas,5,978300760.0
1,1,661,Bloodsport2,3,978302109.0
2,1,914,"Women,The",3,978301968.0
3,1,3408,ReadytoRumble,4,978300275.0
4,1,2355,"Hi-LoCountry,The",5,978824291.0
...,...,...,...,...,...
836478,6041,1356,Grease,5,NaN
836479,6041,1668,Titanic,5,NaN
836480,6041,1833,Mulan,5,NaN
836481,6041,2274,"Bug'sLife,A",5,NaN


In [116]:
ratings=ratings.drop('timestamp', axis=1)
ratings.tail(10)

,user_id,movie_id,movie_name,count
836473,6040,1090,PrerokbeOgnja,3
836474,6040,1094,Palookaville,5
836475,6040,562,NakedinNewYork,5
836476,6040,1096,"Funeral,The",4
836477,6040,1097,Sleepover,4
836478,6041,1356,Grease,5
836479,6041,1668,Titanic,5
836480,6041,1833,Mulan,5
836481,6041,2274,"Bug'sLife,A",5
836482,6041,0,ToyStory,5


In [117]:
ratings['play']=ratings['count']
ratings.drop('count',axis=1)

,user_id,movie_id,movie_name,play
0,1,1193,GoodFellas,5
1,1,661,Bloodsport2,3
2,1,914,"Women,The",3
3,1,3408,ReadytoRumble,4
4,1,2355,"Hi-LoCountry,The",5
...,...,...,...,...
836478,6041,1356,Grease,5
836479,6041,1668,Titanic,5
836480,6041,1833,Mulan,5
836481,6041,2274,"Bug'sLife,A",5


### Finally, preprocessing has finished! 

### 4) make CSR matrix

In [118]:
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
csr_matrix((data, (row, col)), shape=(3, 3)).toarray()

array([[1, 0, 2],
       [0, 0, 3],
       [4, 5, 6]], dtype=int64)

In [119]:
ratings['user_id']

0            1
1            1
2            1
3            1
4            1
          ... 
836478    6041
836479    6041
836480    6041
836481    6041
836482    6041
Name: user_id, Length: 836483, dtype: int64

There are missing values occasionally (not all values are included continuously) 

-> So we will fix the shape with **MAX VALUE, not with nunique()**

In [120]:


# num_user = ratings['user_id'].nunique()
# num_movie = ratings['movie_id'].nunique()
# print(num_user)
# print(num_movie)
# num_add_row=max(ratings.user_id)-num_user
# num_add_column=max(ratings.movie_id)-num_movie

print(max(ratings.user_id))
print(max(ratings.movie_id))

csr_data = csr_matrix((ratings.play, (ratings.user_id, ratings.movie_id)), shape= (max(ratings.user_id)+1,max(ratings.movie_id)+1))

# Check the shape
csr_data.toarray().shape

6041
3952


(6042, 3953)

### 5) organize model and fit
#### als_model = AlternatingleastSquares

In [121]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [122]:
als_model = AlternatingLeastSquares(factors=1000, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

In [123]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [124]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

### 6) pick 
#### one of my favorite
#### another
#### and check out the preference rate the model predicted

In [125]:
mulan = movies_to_idx['Mulan']
crosstar_vector, mulan_vector = als_model.user_factors[my_id], als_model.item_factors[mulan]

print('슝=3')

슝=3


In [126]:
crosstar_vector.shape


(1000,)

In [127]:
mulan_vector.shape

(1000,)

In [128]:
np.dot(crosstar_vector,mulan_vector)

0.78637207

# The preference rate is 0.78!!!

### 7) get recommendation of similar movies(similar_items)

In [129]:
favorite_movie = 'LittlePrincess,A'
fav_movie_id = movies_to_idx[favorite_movie]
similar_movie= als_model.similar_items(fav_movie_id,N=15)
similar_movie

[(259, 1.0),
 (884, 0.677201),
 (33, 0.676116),
 (2576, 0.67330813),
 (3472, 0.6669536),
 (2955, 0.6666617),
 (1470, 0.66664064),
 (1548, 0.6655048),
 (607, 0.66546875),
 (2214, 0.665445),
 (763, 0.6654142),
 (701, 0.66541076),
 (1510, 0.6654091),
 (138, 0.6653943),
 (584, 0.6653556)]

In [130]:
[idx_to_movies[idx[0]] for idx in similar_movie]

['LittlePrincess,A',
 'WildReeds',
 'Babe',
 'RunSilent,RunDeep',
 'DefyingGravity',
 "Felicia'sJourney",
 'Traveller',
 'PicturePerfect',
 'Hellraiser:Bloodline',
 'Impostors,The',
 'ToukiBouki(JourneyoftheHyena)',
 'CelticPride',
 'LateBloomers',
 'UpCloseandPersonal',
 'Aladdin']

In [131]:
def get_similar_movie(movie_name: str):
    movie_id = movies_to_idx[movie_name]
    similar_movies = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movies.get(idx[0]) for idx in similar_movies]
    return similar_movie

In [132]:
get_similar_movie('Babe')

['Babe',
 'FireDownBelow',
 'ToyStory',
 "Bug'sLife,A",
 None,
 'SweetNothing',
 'JungleFever',
 None,
 'FederalHill',
 'She-Devil']

### 8) get recommendation of what i may like

In [133]:
#my_id를 다시 가져옵니다
movies_recommended = als_model.recommend(my_id, csr_data, N=20, filter_already_liked_items=True)
movies_recommended

[(2393, 0.12113064),
 (3260, 0.10504162),
 (3752, 0.09888659),
 (381, 0.09628658),
 (1296, 0.09526387),
 (2431, 0.09523401),
 (1801, 0.09490782),
 (605, 0.0948634),
 (610, 0.09192913),
 (3723, 0.0918338),
 (1892, 0.091820516),
 (832, 0.09053293),
 (2435, 0.08949593),
 (1792, 0.08934063),
 (2718, 0.087962046),
 (3155, 0.08748183),
 (1670, 0.08386505),
 (3200, 0.08343156),
 (3864, 0.08269538),
 (3479, 0.081314266)]

In [135]:
[idx_to_movies.get(i[0]) for i in movies_recommended]

['DryCleaning(Nettoyageàsec)',
 'DogDayAfternoon',
 'AboutAdam',
 'ManofNoImportance,A',
 'BlueJuice',
 'HouseonHauntedHill',
 'Clockwatchers',
 'HomewardBoundII:LostinSanFrancisco',
 'Loaded',
 "BrainThatWouldn'tDie,The",
 'Labyrinth',
 'LottoLand',
 'Rollercoaster',
 'WildManBlues',
 'IronEagle',
 'Bodyguard,The',
 'Twisted',
 'BeautifulPeople',
 None,
 'Jails,Hospitals&Hip-Hop']

In [136]:
caughtup = movies_to_idx['CaughtUp']
explain = als_model.explain(my_id, csr_data, itemid=caughtup)

In [137]:
[(idx_to_movies[i[0]], i[1]) for i in explain[1]]

[('Mulan', 0.08351423854860246),
 ('Titanic', 0.004512514448775712),
 ("Bug'sLife,A", -0.0038371812472055415),
 ('ToyStory', -0.0038850092767695192),
 ('Grease', -0.009861318880009221)]

### Retrospect
1) Sparse Matrix was not made by unique keys, so there are "all-zero" row or column in CSR Matrix
 -> contribution of my favorites were quite lower than i expected